In [1]:
import pandas as pd

answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
                                                                 'recommend','table','picture','video','numword',
                                                                 'thumbsup','thumbsdown','comment','collect','thank',
                                                                 'report','nohelp','oppose'])
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])

In [2]:
from tqdm import tqdm 
tqdm.pandas()

In [3]:
def time_(x):
    temp=x[1:].split('-H')
    return int(temp[0])*24+int(temp[1])

In [4]:
answer_info['atime']=answer_info['atime'].progress_apply(lambda x:time_(x))

100%|██████████| 4513735/4513735 [00:12<00:00, 349987.01it/s]


In [5]:
answer_info=answer_info[['qid','uid','atime']]

In [6]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])

In [7]:
all_data=pd.concat([train[['qid','uid','time']],test[['qid','uid','time']]])

In [8]:
all_data['time']=all_data['time'].progress_apply(lambda x:time_(x))

100%|██████████| 10630880/10630880 [00:29<00:00, 361996.22it/s]


In [9]:
answer_info.columns=['qid','auid','atime']

In [10]:
all_data=pd.merge(all_data,answer_info,on='qid',how='inner')
all_data['sub_time']=all_data['time']-all_data['atime']
all_data=all_data[all_data['sub_time']>0]
len(all_data)#99511676

106035190

In [11]:
all_data=all_data[all_data['uid']!=all_data['auid']]

In [12]:
len(all_data)#99511660

106035169

In [13]:
all_data=all_data.groupby(['uid','qid'])['auid'].agg(['count'])

In [14]:
all_data=all_data.reset_index()

In [15]:
len(all_data)

3855006

In [17]:
all_data.to_hdf('feature_qa_count.h5', key='data')